In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Generate synthetic data for illustration
np.random.seed(42)
n_samples = 100
n_features = 5
X = np.random.rand(n_samples, n_features)  # Random economic indicators
true_weights = np.array([0.5, -0.2, 0.3, 0.1, -0.1])
y = X @ true_weights + np.random.normal(0, 0.1, n_samples)  # Stock prices with noise

# Define the fitness function
def fitness_function(solution, X, y):
    model = LinearRegression()
    model.coef_ = solution
    predictions = X @ model.coef_
    return mean_squared_error(y, predictions)

# Generate the initial population of potential solutions
def generate_initial_population(pop_size, solution_size):
    return np.random.uniform(-1, 1, size=(pop_size, solution_size))

# Create the immune network
def create_immune_network(population, fitness, num_neighbors):
    network = []
    for i, individual in enumerate(population):
        distances = np.linalg.norm(population - individual, axis=1)
        neighbors = np.argsort(distances)[1:num_neighbors+1]  # Exclude self
        network.append(neighbors)
    return network

# Update the immune network
def update_network(network, population, fitness, mutation_rate):
    new_population = np.copy(population)
    for i, neighbors in enumerate(network):
        if np.random.rand() < mutation_rate:
            # Apply mutation with a smaller range
            mutation = np.random.uniform(-0.05, 0.05, population.shape[1])
            new_population[i] += mutation
    return new_population

# Main function implementing Immune Network Theory
def immune_network_theory(solution_size=n_features, pop_size=50, num_neighbors=5, mutation_rate=0.1, generations=50):
    population = generate_initial_population(pop_size, solution_size)
    best_fitness_per_generation = []  # Track the best fitness in each generation

    for generation in range(generations):
        fitness = np.array([fitness_function(ind, X, y) for ind in population])
        network = create_immune_network(population, fitness, num_neighbors)
        new_population = update_network(network, population, fitness, mutation_rate)

        # Evaluate the fitness of the new population
        fitness_new = np.array([fitness_function(ind, X, y) for ind in new_population])

        # Combine the old and new populations
        combined_population = np.vstack((population, new_population))
        combined_fitness = np.hstack((fitness, fitness_new))

        # Select the best individuals
        best_indices = np.argsort(combined_fitness)[:pop_size]
        population = combined_population[best_indices]
        fitness = combined_fitness[best_indices]

        # Track the best fitness of this generation
        best_fitness = np.min(fitness)
        best_fitness_per_generation.append(best_fitness)

        # Stop early if the fitness is good enough
        if best_fitness < 0.01:
            print(f"Optimal solution found in {generation + 1} generations.")
            break

    # Plot the fitness improvement over generations
    plt.figure(figsize=(10, 6))
    plt.plot(best_fitness_per_generation, marker='o', color='blue', label='Best Fitness per Generation')
    plt.xlabel('Generations')
    plt.ylabel('Mean Squared Error')
    plt.title('Fitness Improvement Over Generations')
    plt.grid(True)
    plt.show()

    # Return the best solution found
    best_solution = population[np.argmin(fitness)]
    return best_solution

# Example Usage
best_solution = immune_network_theory()
print("Best solution found (economic indicators weights):", best_solution)
print("Mean Squared Error at best solution:", fitness_function(best_solution, X, y))

# Plot the predicted vs. actual values using the best solution
model = LinearRegression()
model.coef_ = best_solution
predictions = X @ model.coef_

plt.figure(figsize=(8, 6))
plt.scatter(y, predictions, c='blue', label='Predicted vs Actual')
plt.plot([min(y), max(y)], [min(y), max(y)], 'r--', label='Ideal Fit')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Stock Price Prediction with INT')
plt.legend()
plt.grid(True)
plt.show()